<a href="https://colab.research.google.com/github/snipaid-nlg/models/blob/main/news-headline-generation/news-headline-generation-with-german-news-title-gen-mt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Headline Generation with german-news-title-gen-mt5
Evaluating german-news-title-gen-mt5 on the task of generating headlines from journalistic text.

## Installs and Imports

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
from transformers.pipelines.pt_utils import KeyDataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import pandas as pd
from tqdm.auto import tqdm

## Loading and transforming 10kGNAD
We load the [Ten Thousand German News Articles Dataset](https://tblock.github.io/10kGNAD/) from Huggingface. The first sentence of a text (up to the first period) is a headline, so we split the text and use the second part. A Prefix `summarize:` is added because the model was trained on the task of summarization.

In [ ]:
gnad10 = load_dataset("gnad10")
df = pd.DataFrame(gnad10["train"])
df[['headline', 'fulltext']] = df["text"].str.split('.', n=1, expand=True)
df.drop(['label','text'], axis=1, inplace=True)
df['fulltext'] = "summarize: " + df['fulltext']
df.head()

Generating train split:   0%|          | 0/9245 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1028 [00:00<?, ? examples/s]

Dataset gnad10 downloaded and prepared to /root/.cache/huggingface/datasets/gnad10/default/1.1.0/3a8445be65795ad88270af4d797034c3d99f70f8352ca658c586faf1cf960881. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

,headline,fulltext
0,21-Jähriger fällt wohl bis Saisonende aus,summarize: Wien – Rapid muss wohl bis Saisone...
1,"Erfundene Bilder zu Filmen, die als verloren g...",summarize: Wien – Die Filmgeschichte ist ein ...
2,Der frischgekürte CEO Sundar Pichai setzt auf ...,summarize: Die Atmosphäre im Silicon Valley i...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo...",summarize: Moskau – Die russischen Ölproduzen...
4,Estland sieht den künftigen österreichischen P...,summarize: Wien/Tallinn/Pskow – Die Eltern de...


Converting the pandas dataframe to a Huggingface dataset object

In [ ]:
dataset = Dataset.from_pandas(df)

## Loading the model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("aiautomationlab/german-news-title-gen-mt5", use_cache=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("aiautomationlab/german-news-title-gen-mt5")

In [ ]:
template = {
    "num_beams": 5,
    "length_penalty": 1.0,
    "min_new_tokens": 10,
    "no_repeat_ngram_size": 0,
    "repetition_penalty": 1.5,
    "diversity_penalty": 0.0,
    "num_beam_groups": 1.0,
    "do_sample": False,
    "top_k": 50,
    "top_p": 1.0,
    "temperature": 1.0,
    "early_stopping": False,
    "num_return_sequences": 1
}

In [ ]:
pipe = pipeline("text2text-generation", 
                model=model, 
                tokenizer=tokenizer,
                device=0,
                **template)

In [ ]:
for out in tqdm(pipe(KeyDataset(dataset, "fulltext"))):
    print(out)

  0%|          | 0/9245 [00:00<?, ?it/s]

[{'generated_text': 'FC Ingolstadt verzichtet auf Murg'}]
[{'generated_text': 'The Forbidden Room von Guy Maddin und Evan Johnson'}]
[{'generated_text': 'Die Zeit der ausgefahrenen Ellbogen ist vorbei'}]
[{'generated_text': 'Russland will Ölproduktion auf dem Niveau von Jänner einfrieren'}]
[{'generated_text': 'Estnischer Präsident Van der Bellen ist tot'}]
[{'generated_text': 'Trotec übernimmt zwei US-Firmen'}]
[{'generated_text': 'Heracles empfängt Leidens FC Utrecht'}]
[{'generated_text': 'Wie geht es weiter beim ORF-Geschäftsbau?'}]
[{'generated_text': 'Pakistanische Luftwaffe tötet 43 Extremisten in Nordwaziristan'}]
[{'generated_text': 'Bayern hofft auf einen Sahnetag'}]
[{'generated_text': 'Serie A: Italien gewinnt in San Siro mit 5:1'}]
[{'generated_text': 'OMV kauft 66 Diskonttankstellen in Hofer'}]
[{'generated_text': 'Remake von Final Fantasy 7 auf der PS4'}]


KeyboardInterrupt: ignored